# Creating your first Experiment in CRIPT 

## Create a CRIPT account and retrieve your API token
* To create a user account on CRIPT please login through the [web application](https://criptapp.herokuapp.com/) with an [ORCID](https://orcid.org/).
* After login, an API token can be retrieved from the [account settings page](https://criptapp.herokuapp.com/settings/).
* We recommend storing your API token as an evironment variable. Once connected, you should be able to view the API easily here: https://criptapp.herokuapp.com/api/

## 1. Get your user info from `User node`
* To create various assests in CRIPT (Collections, Experiments etc.) you will need to query your User node for it's url. 

## 2. Find or create a `Group node`
* Similar to the User node, many CRIPT assests require a Group to be defined upon creation.

## 3. Create a `Collection node`
* A Collection node is an organizational tool, you can think of it as a 'folder' on your computer. It just a way to group experiments together. 
* Examples - a series of kinetic experiments on a reaction, a library of polyesters, all the experiments that make up a publication
* Example of nested Collections - An example of that may be you find a new catalyst that can polymerize styrene. You could have a parent collection which holds several other collections child collections with each child collection deticated to a different aspect of the new catalyst reaseach (such as kinetics study, monomer scope, diblock polymer synthesis, etc.). 

## Next, lets make our `Experiment node`.
* The `Experiment node` is just a grouping of `Material nodes`, `Process nodes` and `Data nodes`. You can think of an experiment as a page in the notebook, except it doesn't hold any of the data it just reference the nodes that do. The reason we break out the `Material nodes`, `Process nodes` and `Data nodes` and don't just group them into the `Experiment node` is because a single `Material` may be referenced in multiple experiments and this allows us to just refenence the `Material node` instead of duplicating the data. Additionally, this seperated data model is more flexable to cases where someone makes a material in one experiment and uses it or studies in a different experiment. There is also huge benfits in terms of search perferomance and reduced memory usage, but that's beyond the scope of this tutorial.

## The `Material node` contains identity and property data for a chemical. 
* This can be a polymer, a small molecule, or a mixture of multiple components. CRIPT is already prepopulated with the most common small molecules (including some property data). You can of course add a material if it doesn't already exist in the database, or reference an existing material and add new data, or completely copy a whole material and modify it to suit your own requirements. In terms of best practices (for small molecules), using existing materials is always prefered, followed by referencing, and coping as the least prefered method as duplicates of the same material hinders the effectiveness of the database. When it comes to polymers, polymers should aways be treated as a new material as the molecular weight distribution, composition, topolology is never perfectly the same.

In [18]:
import os

# this assumes you have downloaded the CRIPT python package
import sys
sys.path.append('../../')
from cript import * 

# Initialize API session (i.e. login)
session = connect.API(url="https://criptapp.herokuapp.com", token=os.environ['CRIPT_API_TOKEN_2'])
session


Connection to the API was successful!



Connected to https://criptapp.herokuapp.com/api

In [19]:
# 1. Cache your user info
query_params = {'username':'cborg'}
user_info = session.get(User, query_params)
user_info.url

'https://criptapp.herokuapp.com/api/user/a9e30833-c66b-488e-8c6b-d6ea305a3af2/'

In [10]:
# 2a. Find a group by name
query_params = {'name':'Tutorials'}
data = session.get(Group, query_params)

# 2b. create a new group (must have user url)
group = nodes.Group(name='Tutorial group', users=[data.url])
session.save(group)

{
    "url": "https://criptapp.herokuapp.com/api/group/8889d011-3e02-4bfd-bea2-c600a1f9de6a/",
    "uid": "8889d011-3e02-4bfd-bea2-c600a1f9de6a",
    "name": "Tutorials",
    "users": [
        "https://criptapp.herokuapp.com/api/user/a9e30833-c66b-488e-8c6b-d6ea305a3af2/"
    ],
    "public": false,
    "created_at": null,
    "updated_at": null
}

In [16]:
# 3. Create a new collection (must have group url)
collection_node = nodes.Collection(name='Tutorials', group=group.url)
session.save(collection_node)

Collection node has been saved to the database.


In [45]:
# 4. 
expt_node = nodes.Experiment(name="Anionic Polymerization of Styrene with SecBuLi",collection=collection_node,group=group)

sec_data = nodes.Data(
    name="Crude SEC of polystyrene",
    type="sec_trace",
    group=group,
    experiment=expt_node,
    sample_prep="5 mg of polymer in 1 ml of THF, filtered 0.45um pores.",
)


conditions =[
        nodes.Condition("temperature", value=100, unit='degC'),
        # nodes.Condition("time", value=60, unit='min'),
        nodes.Condition("solvent", value='THF'),
        # nodes.Condition("folw_rate", value=1, unit='ml/min')
    ]

iden=nodes.Identifier(key='CAS', value='123')

In [67]:
# update a collection
query_params = {'name':'my_first_collection'}
collection = session.get(Collection, query_params)
print(collection, type(collection))
collection.notes = 'I added a note'
session.save(collection)
print(collection, type(collection))

{
    "url": "https://criptapp.herokuapp.com/api/collection/35/",
    "group": "https://criptapp.herokuapp.com/api/group/34/",
    "name": "my_first_collection",
    "notes": null,
    "experiments": [],
    "inventories": [],
    "citations": [],
    "created_at": "2022-04-04T20:37:17.919861Z",
    "updated_at": "2022-04-04T20:37:17.919883Z",
    "public": false
} <class 'cript.nodes.Collection'>
Collection node has been saved to the database.
{
    "url": "https://criptapp.herokuapp.com/api/collection/35/",
    "group": "https://criptapp.herokuapp.com/api/group/34/",
    "name": "my_first_collection",
    "notes": "I added a note",
    "experiments": [],
    "inventories": [],
    "citations": [],
    "created_at": "2022-04-04T20:37:17.919861Z",
    "updated_at": "2022-04-04T20:37:22.334055Z",
    "public": false
} <class 'cript.nodes.Collection'>


In [62]:
# delete a collection
query = session.delete(collection)
print(query, type(query))

Collection node has been deleted from the database.
None <class 'NoneType'>
